In [1]:
# импортируем нужные библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# считаем данные в DataFrame
data = pd.read_csv('Lab2Part1Train.csv', index_col=0)
data.head(10)

,x_1,x_2,x_3,x_4,x_5,answer
1,0,0,0,0,1,k_1
2,1,0,0,1,1,k_1
3,0,1,0,0,0,k_1
4,1,0,0,0,1,k_1
5,1,1,0,0,0,k_1
6,0,0,0,1,0,k_2
7,1,1,0,1,1,k_2
8,0,1,0,1,0,k_2
9,1,0,1,0,1,k_2
10,0,1,0,1,1,k_2


In [3]:
# На вход функция принимает, датафрейм, имя столбца для которого считаем энтропию, коэффициенты
def entropy (data, columnName,koeff): 
    hist = np.bincount(data[columnName].to_list())
    ps = hist / len(data[columnName])
    entr = -np.sum([k * p * np.log2(p) for p,k in zip(ps,koeff) if p > 0])
    return entr

In [4]:
# на вход подаём датафрейм, для которого нужно найти вершину
def findNode (data):
#     сохраняем заголовки таблицы, первый заголовок считаем минимальным  
    cols = data.columns 
    columnMin = cols[0]
    summ_prev = float('inf') 
#     проходимся по индексам столбцов от 0 до предпоследнего
    for i in range(len(cols)-1):
#         hist - количество элементов первого, второго,...,n-го типа в столбце
        hist = np.bincount(data[cols[i]].to_list())
#         ps - вероятность выбора элемента первого, второго, ..., n-го типа 
        ps = hist / len(data[cols[i]])
#         список классов
        us = data['answer'].unique()
    
        s_shannon = np.sum([entropy(data.loc[data['answer'] == u],cols[i],ps) for u in us])
           
        if s_shannon < summ_prev: 
            columnMin = cols[i]
            summ_prev = s_shannon
        print("Для ",cols[i], "критерий Шеннона =", s_shannon)
#     Делим датафрейм на два датафрейма. Правый - все строки для которых, у столбца с максимальной суммой (по D-критерию), 
#     стоят значения - "1".Левый - для тех строк, у которых значения - "0". 
#     Одновременно избавляемся от найденного столбца, он нам больше не нужен.
    dataRight = data.loc[data[columnMin] == 1].drop(columns = [columnMin], axis = 1)
    dataLeft = data.loc[data[columnMin] == 0].drop(columns = [columnMin], axis = 1) 
    return columnMin, dataLeft , dataRight 

In [5]:
# класс, чтобы хранить наше дерево
class Node:
    def __init__(self, id: int, name : str, left=None, right=None):
        self.id = id 
        self.name = name
        self.left  = left
        self.right = right
    def __str__(self):
        return self.name

In [6]:
# глобальная переменная counter, чтобы присваивать уникальный id вершинам
counter = 0 
# Рекурсивная функция для построения дерева, на вход подаем датафрейм и дерево. 
# Для первого запуска - дерево не нужно, достаточно только датафрейма
def tree_construct(dataFrame, tree = None): 
    global counter
#     если интервал чистый (т.е. все значения принадлежат одному из классов)
    if len(dataFrame['answer'].unique())<=1:
# обзываем лист дерева именем класса, в котором сейчас находимся
        nodeName = dataFrame['answer'].values[0]
# левого и правого поддерева - не будет
        left = None
        right = None
#     Иначе вызываем findNode. В параметре передаем текущий датафрейм, на выходе получаем полученную 
#     вершину по D-критерию и два датафрейма - левый и правый
    else:
        nodeName, dL, dR = findNode(dataFrame)
#         строим дерево для левого и правого датафреймов
        left = tree_construct(dL, tree)
        right = tree_construct(dR, tree)
    tree = Node(counter, nodeName,left, right)
#     обновляем счетчик для вершин
    counter += 1
    return(tree)

In [7]:
tree = tree_construct(data)

Для  x_1 критерий Шеннона = 0.9709505944546686
Для  x_2 критерий Шеннона = 0.9709505944546686
Для  x_3 критерий Шеннона = 0.27822679021664815
Для  x_4 критерий Шеннона = 0.7219280948873623
Для  x_5 критерий Шеннона = 0.9536322181636243
Для  x_1 критерий Шеннона = 0.8854404708965871
Для  x_2 критерий Шеннона = 0.8854404708965871
Для  x_4 критерий Шеннона = 0.37245533316965795
Для  x_5 критерий Шеннона = 0.9806646371464887
Для  x_1 критерий Шеннона = 0.3867668746754797
Для  x_2 критерий Шеннона = 0.3867668746754797
Для  x_5 критерий Шеннона = 0.5
Для  x_2 критерий Шеннона = 0.0
Для  x_5 критерий Шеннона = 0.0


In [8]:

import pydot

graph = pydot.Dot("my_graph", graph_type="graph")
    
# рекурсивная функция для добавления в граф вершин дерева    
def print_tree_node(tree):
    if tree == None: return
    print_tree_node(tree.left)
    print_tree_node(tree.right)
    if tree.name == "k_1" or tree.name == "k_2":
        graph.add_node(pydot.Node(tree.id,label=tree.name,shape="rectangle"))
    else:
        graph.add_node(pydot.Node(tree.id,label=tree.name,shape="circle")),

# рекурсивная функция для добавления в граф связей между вершинами
def print_tree_edge(tree):
    if tree == None: return
    if tree.left != None: 
        graph.add_edge(pydot.Edge(tree.id, tree.left.id , color="blue"))
    if tree.right != None:
        graph.add_edge(pydot.Edge(tree.id, tree.right.id , color="blue"))
    print_tree_edge(tree.left)
    print_tree_edge(tree.right),
    
print_tree_node(tree)
print_tree_edge(tree)


In [9]:
# визуализируем
graph.write_png("output2.png")
from IPython.display import IFrame
IFrame('output2.png', width=800, height=400)

In [10]:
# считаем данные в DataFrame
dataTest = pd.read_csv('Lab2Part1Test.csv', index_col=0)
dataTest.head(10)

,x_1,x_2,x_3,x_4,x_5
1,0,0,0,0,0
2,0,0,0,0,1
3,0,0,0,1,0
4,0,0,0,1,1
5,0,0,1,0,0
6,0,0,1,0,1
7,0,0,1,1,0
8,0,0,1,1,1
9,0,1,0,0,0
10,0,1,0,0,1


In [11]:
answers = []
def predictFunc(tree,i): 
    if tree == None: return
    if tree.name == "k_1" or tree.name == "k_2":
        answers.append(tree.name)
    elif dataTest[tree.name].values[i] == 0: 
        predictFunc(tree.left, i)
    else: 
        predictFunc(tree.right, i)
        
for i in range(len(dataTest.index)):       
    predictFunc(tree,i)

dataTest.insert(len(dataTest.columns), "Answer", answers, False)

In [12]:
dataTest.head(10)

,x_1,x_2,x_3,x_4,x_5,Answer
1,0,0,0,0,0,k_1
2,0,0,0,0,1,k_1
3,0,0,0,1,0,k_2
4,0,0,0,1,1,k_2
5,0,0,1,0,0,k_2
6,0,0,1,0,1,k_2
7,0,0,1,1,0,k_2
8,0,0,1,1,1,k_2
9,0,1,0,0,0,k_1
10,0,1,0,0,1,k_1
